In [148]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from flask import Flask, render_template, request

Read the dataset

In [149]:
df = pd.read_csv('marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv')

In [150]:
# df = df.drop(['Brand Name', 'Asin','Upc Ean Code', 'List Price','Model Number','Quantity','Sku','Product Dimensions','Stock','Product Details','Dimensions','Color','Ingredients','Direction To Use','Size Quantity Variant','Product Description'], axis=1)

Create an instance of the Elasticsearch client

In [151]:
df.fillna("N/A", inplace = True)
print(df.head(19))

                             Uniq Id  \
0   4c69b61db1fc16e7013b43fc926e502d   
1   66d49bbed043f5be260fa9f7fbff5957   
2   2c55cae269aebf53838484b0d7dd931a   
3   18018b6bc416dab347b1b7db79994afa   
4   e04b990e95bf73bbe6a3fa09785d7cd0   
5   f8c32a45e507a177992973cf0d46d20c   
6   40d3cd16b41970ae6872e914aecf2c8e   
7   bc178f33a04dbccefa95b165f8b56830   
8   cc2083338a16c3fe2f7895289d2e98fe   
9   69828c5570dcdbd1609216d33fbad0db   
10  7b0746d8afc8462ba17f8a763d9d5f1e   
11  c5f4c94653a3befd8dd16adf2914c04e   
12  615ba903c134f439eaf8cdd1678ceb5c   
13  82c86a4d24dce5e14303033d7b658b78   
14  1ecccb43e0f5c0162218371916ffa553   
15  d1ac6d805fcdf75f9e00af611dd03ce4   
16  d82015ffbf4cd54b0edfb890002971f3   
17  39f1b8a2129315da0288cd058b6b6086   
18  4af0833f20a8421788fc04577b0ef1c5   

                                         Product Name Brand Name Asin  \
0   DB Longboards CoreFlex Crossbow 41" Bamboo Fib...        N/A  N/A   
1   Electronic Snap Circuits Mini Kits Classpack, ...

In [152]:
es = Elasticsearch(['http://localhost:9200'],request_timeout=3600)  # 连接本地9200端口

Create a mapping for the index

In [153]:
doc_mapping = {
      "properties": {
        "product_name": {
          "type": "text",
          'store': True
        },
        "brand_name": {
          "type": "text",
          'store': True
        },
        "asin": {
          "type": "text",
          'store': True
        },
        "category": {
          "type": "text",
          'store': True
        },
        "upc_ean_code": {
          "type": "text",
          'store': True
        },
        "list_price": {
          "type": "text",
          'store': True
        },
        "selling_price": {
          "type": "text",
          'store': True,
        'fielddata':True
        },
        "quantity": {
          "type": "text",
          'store': True
        },
        "model_number": {
          "type": "text",
          'store': True
        },
        "about_product": {
          "type": "text",
          'store': True
        },
        "product_specification": {
          "type": "text",
          'store': True
        },
        "technical_details": {
          "type": "text",
          'store': True
        },
        "shipping_weight": {
          "type": "text",
          'store': True
        },
        "product_dimensions": {
          "type": "text",
          'store': True
        },
        "image": {
          "type": "text",
          'store': True
        },
        "variants": {
          "type": "text",
          'store': True
        },
        "sku": {
          "type": "text",
          'store': True
        },
        "product_url": {
          "type": "text",
          'store': True
        },
        "stock": {
          "type": "text",
          'store': True
        },
        "product_details": {
          "type": "text",
          'store': True
        },
        "dimensions": {
          "type": "text",
          'store': True
        },
        "color": {
          "type": "text",
          'store': True
        },
        "ingredients": {
          "type": "text",
          'store': True
        },
        "direction_to_use": {
          "type": "text",
          'store': True
        },
        "is_amazon_seller": {
          "type": "text",
          'store': True
        },
        "size_quantity_variant": {
          "type": "text",
          'store': True
        },
        "product_description": {
          "type": "text",
          'store': True
        }
      }
    }


Create the index

In [154]:
# es.indices.create(index="amazon_products", body=mapping)
if es.indices.exists(index="amazon_products"):
    print('Index {} exists. removing it...'.format("amazon_products"))
    es.indices.delete(index="amazon_products")

es.options(ignore_status=[400]).indices.create(
            index="amazon_products",
            mappings=doc_mapping)

Index amazon_products exists. removing it...


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'amazon_products'})

Index the data

In [155]:
for index, row in df.iterrows():
    document = {
        "product_name": ''.join(row["Product Name"]),
        "brand_name": ''.join(row["Brand Name"]),
        "asin": ''.join(row["Asin"]),
        "category": ''.join(row["Category"]),
        "upc_ean_code": ''.join(row["Upc Ean Code"]),
        "list_price": ''.join(row["List Price"]),
        "selling_price": row["Selling Price"].replace("$",""),
        "quantity": ''.join(row["Quantity"]),
        "model_number": ''.join(row["Model Number"]),
        "about_product": row["About Product"].replace("Make sure this fits by entering your model number.",""),
        "product_specification": ''.join(row["Product Specification"]),
        "technical_details": ''.join(row["Technical Details"]),
        "shipping_weight": ''.join(row["Shipping Weight"]),
        "product_dimensions": ''.join(row["Product Dimensions"]),
        "image": ''.join(row["Image"]),
        "variants": ''.join(row["Variants"]),
        "sku": ''.join(row["Sku"]),
        "product_url": ''.join(row["Product Url"]),
        "stock": ''.join(row["Stock"]),
        "product_details": ''.join(row["Product Details"]),
        "dimensions": ''.join(row["Dimensions"]),
        "color": ''.join(row["Color"]),
        "ingredients": ''.join(row["Ingredients"]),
        "direction_to_use": ''.join(row["Direction To Use"]),
        "is_amazon_seller": ''.join(row["Is Amazon Seller"]),
        "size_quantity_variant": ''.join(row["Size Quantity Variant"]),
        "product_description": ''.join(row["Product Description"])
    }
    res = es.index(index="amazon_products", document=document)

Create the Flask app

In [130]:

query = {
        "multi_match" : {
        "query":    "phone", 
        "fields": [ "product_name", "about_product" ] 
        }
    }

# 查询name包含python关键字的数据
es.search(index="amazon_products",query=query)

ObjectApiResponse({'took': 176, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 88, 'relation': 'eq'}, 'max_score': 9.048491, 'hits': [{'_index': 'amazon_products', '_id': 'Qm1k64QBqO1y_XCjGNNQ', '_score': 9.048491, '_source': {'product_name': 'Fisher-Price Musical Smart Phone', 'brand_name': 'N/A', 'asin': 'N/A', 'category': 'Toys & Games | Baby & Toddler Toys | Musical Toys', 'upc_ean_code': 'N/A', 'list_price': 'N/A', 'selling_price': '$4.99', 'quantity': 'N/A', 'model_number': 'N/A', 'about_product': 'Say “hello” to the updated Smart Phone design | Press button for realistic sounds & fun music | Icons rotate! | Who’s that? “Selfie” mirror invites self-discovery | Fine-motor-skill building activities', 'product_specification': 'N/A', 'technical_details': 'N/A', 'shipping_weight': 'N/A', 'product_dimensions': 'N/A', 'image': 'https://images-na.ssl-images-amazon.com/images/I/31VRsgWVSIL.jpg|https://images-na.ssl-imag

22
